# Touché 25 RAD Examples
## Example: Retrieve arguments with dense retrieval

#### 1. Install requirements

In [1]:
import json

%pip install elasticsearch~=8.15.1
%pip install sentence-transformers
%pip install xformers

  Using cached elastic_transport-8.17.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 3.0 MB/s eta 0:00:00
Using cached elastic_transport-8.17.1-py3-none-any.whl (64 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached xformers-0.0.29.post3.tar.gz (8.5 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      ERROR: Can not execute `setup.py` since setuptools is not available in the build environment.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for d

#### 2. Connect to Elasticsearch

In [2]:
from elasticsearch import Elasticsearch

# for local Elasticsearch instance
# elasticsearch_url = "http://localhost:9200"

# hosted index
elasticsearch_url = "https://touche25-rad.webis.de/arguments/"

es_client = Elasticsearch(elasticsearch_url, retry_on_timeout=True)
index_name = "claimrev"

query = "pineapple belongs on pizza"

#### 3. Embed a query

##### 3a. Use Stella embeddings

In [4]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(
    "dunzhang/stella_en_400M_v5", trust_remote_code=True
)

query_embedding = embedding_model.encode(query, prompt_name="s2p_query")

text_field = "text_embedding_stella"
supports_field = "supports_embedding_stella"
attacks_field = "attacks_embedding_stella"

RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
module 'sklearn.metrics._dist_metrics' has no attribute 'DistanceMetric32'

#### 4. Retrieve arguments

##### Search based on the text field

In [71]:
# Search based on the argument text
resp = es_client.search(
    index=index_name,
    knn={
        "field": text_field,
        "query_vector": query_embedding,
        "k": 10,
        "num_candidates": 100,
    },
)

for hit in resp["hits"]["hits"]:
    # Remove embedding vectors for displaying purposes
    del hit["_source"]["attacks_embedding_stella"]
    # del hit["_source"]["attacks_embedding_colbert"]
    del hit["_source"]["supports_embedding_stella"]
    # del hit["_source"]["supports_embedding_colbert"]
    del hit["_source"]["text_embedding_stella"]
    # del hit["_source"]["text_embedding_colbert"]

    print(json.dumps(hit["_source"], indent=2))

{
  "id": "10104.845",
  "topic": "Does pineapple belong on pizza?",
  "tags": [
    "Food",
    "Pizza",
    "Hawaiian",
    "Pineapple"
  ],
  "attacks": "The nature of pizza prevents pineapple from being on it.",
  "supports": "The nature of pizza does not prevent pineapple from being on it.",
  "text": "Nothing in the definition of the word 'pizza' says that pineapple should not or does not belong on it",
  "references": [
    "https://www.merriam-webster.com/dictionary/pizza"
  ],
  "original": "attacks"
}
{
  "id": "10104.1196",
  "topic": "Does pineapple belong on pizza?",
  "tags": [
    "Food",
    "Pizza",
    "Hawaiian",
    "Pineapple"
  ],
  "attacks": "Torture is right. In today's day and age, we should not have moved beyond this kind of barbarism.",
  "supports": "Torture is wrong. In today's day and age, we should have moved well beyond this kind of barbarism. It's cruel to a tropical fruit to be stuck on top of a pizza and be shoved into an oven.",
  "text": "Being a t

##### Search based on argument supports

In [72]:
import json

# Search based on the support
resp = es_client.search(
    index=index_name,
    knn={
        "field": supports_field,
        "query_vector": query_embedding,
        "k": 10,
        "num_candidates": 100,
    },
)

for hit in resp["hits"]["hits"]:
    # Remove embedding vectors for displaying purposes
    del hit["_source"]["attacks_embedding_stella"]
    # del hit["_source"]["attacks_embedding_colbert"]
    del hit["_source"]["supports_embedding_stella"]
    # del hit["_source"]["supports_embedding_colbert"]
    del hit["_source"]["text_embedding_stella"]
    # del hit["_source"]["text_embedding_colbert"]

    print(json.dumps(hit["_source"], indent=2))

{
  "id": "10104.349",
  "topic": "Does pineapple belong on pizza?",
  "tags": [
    "Food",
    "Pizza",
    "Hawaiian",
    "Pineapple"
  ],
  "attacks": "Pineapple does not belong on pizza.",
  "supports": "Pineapple belongs on pizza.",
  "text": "Pineapples make pizza aesthetically pleasing.",
  "references": [],
  "original": "supports"
}
{
  "id": "10104.425",
  "topic": "Does pineapple belong on pizza?",
  "tags": [
    "Food",
    "Pizza",
    "Hawaiian",
    "Pineapple"
  ],
  "attacks": "Pineapple does not belong on pizza.",
  "supports": "Pineapple belongs on pizza.",
  "text": "It's no coincidence that pineapple and pizza both start with pi.",
  "references": [],
  "original": "supports"
}
{
  "id": "10104.2225",
  "topic": "Does pineapple belong on pizza?",
  "tags": [
    "Food",
    "Pizza",
    "Hawaiian",
    "Pineapple"
  ],
  "attacks": "Pineapple does not belong on pizza.",
  "supports": "Pineapple belongs on pizza.",
  "text": "Pineapple is cheaper than other toppi

##### Search based on the argument attacks

In [73]:
import json

# Search based on the support
resp = es_client.search(
    index=index_name,
    knn={
        "field": attacks_field,
        "query_vector": query_embedding,
        "k": 10,
        "num_candidates": 100,
    },
)

for hit in resp["hits"]["hits"]:
    # Remove embedding vectors for displaying purposes
    del hit["_source"]["attacks_embedding_stella"]
    # del hit["_source"]["attacks_embedding_colbert"]
    del hit["_source"]["supports_embedding_stella"]
    # del hit["_source"]["supports_embedding_colbert"]
    del hit["_source"]["text_embedding_stella"]
    # del hit["_source"]["text_embedding_colbert"]

    print(json.dumps(hit["_source"], indent=2))

{
  "id": "10104.715",
  "topic": "Does pineapple belong on pizza?",
  "tags": [
    "Food",
    "Pizza",
    "Hawaiian",
    "Pineapple"
  ],
  "attacks": "Pineapple belongs on pizza.",
  "supports": "Pineapple does not belong on pizza.",
  "text": "While some benefit from pineapples being on pizza, other living beings suffer from it.",
  "references": [],
  "original": "attacks"
}
{
  "id": "10104.885",
  "topic": "Does pineapple belong on pizza?",
  "tags": [
    "Food",
    "Pizza",
    "Hawaiian",
    "Pineapple"
  ],
  "attacks": "Pineapple belongs on pizza.",
  "supports": "Pineapple does not belong on pizza.",
  "text": "The nature of pizza prevents pineapple from being on it.",
  "references": [],
  "original": "attacks"
}
{
  "id": "10104.1065",
  "topic": "Does pineapple belong on pizza?",
  "tags": [
    "Food",
    "Pizza",
    "Hawaiian",
    "Pineapple"
  ],
  "attacks": "Pineapple belongs on pizza.",
  "supports": "Pineapple does not belong on pizza.",
  "text": "Becaus